<a href="https://colab.research.google.com/github/HARIPRIYA02/DPOptimizers---HS/blob/main/WRN-16-4-%20custom%20private%20rmsprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opacus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 11.9 MB/s eta 0:00:00


In [5]:
import warnings
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import torch.nn as nn
import torch.optim as optim
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from opacus.validators import ModuleValidator
from tqdm import tqdm
import numpy as np

warnings.simplefilter("ignore")

# Hyperparameters
MAX_GRAD_NORM = 1.2
EPSILON = 3.0
DELTA = 1e-5
NOISE_MULTIPLIER = 1.1
EPOCHS = 50
LR = 8e-4
BATCH_SIZE = 4096
MAX_PHYSICAL_BATCH_SIZE = 128
CIFAR10_MEAN = [0.4914, 0.4822, 0.4465]
CIFAR10_STD_DEV = [0.2023, 0.1994, 0.2010]
NUM_GROUPS = 40

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV)
])

# Load datasets
DATA_ROOT = "~/.local/data"
train_dataset = CIFAR10(root=DATA_ROOT, train=True, download=True, transform=transform)
test_dataset = CIFAR10(root=DATA_ROOT, train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define the weight-standardized convolutional layer
class WSConv2d(nn.Conv2d):
    def forward(self, x):
        mean = self.weight.mean(dim=[1, 2, 3], keepdim=True)
        std = self.weight.std(dim=[1, 2, 3], keepdim=True) + 1e-5
        weight = (self.weight - mean) / std
        return nn.functional.conv2d(x, weight, self.bias, self.stride, self.padding, self.dilation, self.groups)

# Define ResNet-20 with Group Normalization and Weight Standardization
class BasicBlockGN(nn.Module):
    def __init__(self, in_planes, planes, stride=1, num_groups=2):
        super(BasicBlockGN, self).__init__()
        self.conv1 = WSConv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.gn1 = nn.GroupNorm(num_groups, planes)
        self.conv2 = WSConv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.gn2 = nn.GroupNorm(num_groups, planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                WSConv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(num_groups, planes)
            )

    def forward(self, x):
        out = nn.SiLU()(self.gn1(self.conv1(x)))
        out = self.gn2(self.conv2(out))
        out += self.shortcut(x)
        out = nn.SiLU()(out)
        return out

class ResNet20GN(nn.Module):
    def __init__(self, num_classes=10, num_groups=2):
        super(ResNet20GN, self).__init__()
        self.in_planes = 16
        self.conv1 = WSConv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.gn1 = nn.GroupNorm(num_groups, 16)
        self.layer1 = self._make_layer(BasicBlockGN, 16, 3, stride=1, num_groups=num_groups)
        self.layer2 = self._make_layer(BasicBlockGN, 32, 3, stride=2, num_groups=num_groups)
        self.layer3 = self._make_layer(BasicBlockGN, 64, 3, stride=2, num_groups=num_groups)
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride, num_groups):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, num_groups))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = nn.SiLU()(self.gn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = nn.AvgPool2d(8)(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

class WideBasicGN(nn.Module):
    def __init__(self, in_planes, planes, dropout_rate, stride=1, num_groups=2):
        super(WideBasicGN, self).__init__()
        self.conv1 = WSConv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.gn1 = nn.GroupNorm(num_groups, planes)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.conv2 = WSConv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.gn2 = nn.GroupNorm(num_groups, planes)

        # Ensure the shortcut connection has matching dimensions
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                WSConv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(num_groups, planes)
            )

    def forward(self, x):
        out = nn.SiLU()(self.gn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.gn2(self.conv2(out))
        out += self.shortcut(x)  # Add the shortcut, now with matching dimensions
        out = nn.SiLU()(out)
        return out


class WRN16_4_GN(nn.Module):
    def __init__(self, num_classes=10, dropout_rate=0.3, num_groups=2):
        super(WRN16_4_GN, self).__init__()
        self.in_planes = 16
        self.conv1 = WSConv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.gn1 = nn.GroupNorm(num_groups, 16)
        self.layer1 = self._make_layer(WideBasicGN, 16, 3, dropout_rate, stride=1, num_groups=num_groups)
        self.layer2 = self._make_layer(WideBasicGN, 32, 3, dropout_rate, stride=2, num_groups=num_groups)
        self.layer3 = self._make_layer(WideBasicGN, 64, 3, dropout_rate, stride=2, num_groups=num_groups)
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, block, planes, num_blocks, dropout_rate, stride, num_groups):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride, num_groups))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = nn.SiLU()(self.gn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = nn.AvgPool2d(8)(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Initialize model, optimizer, and privacy engine
#model = ResNet20GN(num_classes=10)
model = WRN16_4_GN(num_classes=10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModuleValidator.fix(model)
model = model.to(device)
optimizer = optim.RMSprop(model.parameters(), lr=LR, momentum=0.9, alpha=0.99)
ModuleValidator.validate(model, strict=False)

privacy_engine = PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM
)

# Adaptive learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS * len(train_loader), eta_min=8e-6)

# Loss function
criterion = nn.CrossEntropyLoss()

# Training and testing functions
def train(model, train_loader, optimizer, epoch, device):
    model.train()
    losses = []
    top1_acc = []

    with BatchMemoryManager(data_loader=train_loader, max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, optimizer=optimizer) as memory_safe_data_loader:
        for batch_idx, (data, target) in tqdm(enumerate(memory_safe_data_loader), total=len(memory_safe_data_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            losses.append(loss.item())
            _, predicted = torch.max(output.data, 1)
            correct = predicted.eq(target).sum().item()
            top1_acc.append(correct / target.size(0))

    avg_loss = np.mean(losses)
    avg_acc = np.mean(top1_acc) * 100
    return avg_loss, avg_acc

# Test accuracy will be multiplied by 100 for percent representation - for e.g. 0.60%  is actually 60%
def test(model, test_loader, device):
    model.eval()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            # Calculate accuracy
            acc = np.mean(preds == labels)
            losses.append(loss.item())
            top1_acc.append(acc)

    avg_loss = np.mean(losses)
    avg_accuracy = np.mean(top1_acc)
    return avg_loss, avg_accuracy

# Train and validate the model
for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train(model, train_loader, optimizer, epoch, device)
    test_loss, test_accuracy = test(model, test_loader, device)
    print(f'Epoch: {epoch}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, '
          f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

    scheduler.step(test_loss)

# Print final epsilon after training
final_epsilon = privacy_engine.get_epsilon(DELTA)
print(f"Final epsilon after {EPOCHS} epochs: {final_epsilon:.2f}")


Files already downloaded and verified
Files already downloaded and verified


398it [00:57,  6.89it/s]


Epoch: 1, Train Loss: 2.1484, Train Accuracy: 17.72%, Test Loss: 2.0449, Test Accuracy: 0.25%


400it [00:55,  7.26it/s]


Epoch: 2, Train Loss: 1.9746, Train Accuracy: 25.75%, Test Loss: 1.9571, Test Accuracy: 0.28%


...

Epoch: 48, Train Loss: 1.6474, Train Accuracy: 51.71%, Test Loss: 2.7002, Test Accuracy: 0.47%


398it [00:55,  7.21it/s]


Epoch: 49, Train Loss: 1.6477, Train Accuracy: 51.78%, Test Loss: 2.3765, Test Accuracy: 0.50%


397it [00:54,  7.23it/s]


Epoch: 50, Train Loss: 1.6137, Train Accuracy: 52.59%, Test Loss: 2.4589, Test Accuracy: 0.49%
Final epsilon after 50 epochs: 3.00


In [6]:
top1_acc=test(model,test_loader, device)
top1_acc

(2.4588874181111655, 0.4944308570704277)